In [2]:
import pandas as pd
import numpy as np
import math
from statistics import stdev
fayl = pd.read_excel('/Users/ctlewis/Documents/Projects/Newberry maybe/Data/Phases/Phases_new_useful/Lewis_051321_fayalite.xlsx',
                    sheet_name='filtered')
fayl.head()
#spreadsheet = spreadsheet[spreadsheet['Li7'] != 0]
#spreadsheet.loc[:,cols] = spreadsheet.loc[:,cols].ffill()

,Comment,sio2,al2o3,mno,feo,na2o,cao,tio2,cr2o3,nio,mgo,total
0,NB09_07,33.131524,0.219207,2.578288,43.622129,0.031315,1.513570,0.104384,0.020877,0.052192,18.726514,100
1,NB09_07,32.867279,0.323456,2.566778,44.636895,0.000000,1.387730,0.125209,0.041736,0.020868,18.030050,100
2,NB09_07,32.043232,0.257334,3.067422,46.587751,0.020587,1.348430,0.133814,0.020587,0.041173,16.500257,100
3,NB09_07,31.969415,0.247985,2.965489,46.683199,0.041331,1.363918,0.103327,0.020665,0.010333,16.594338,100
4,NB09_07,33.049881,0.362958,2.416260,43.648242,0.031111,1.389609,0.145183,0.031111,0.041481,18.873794,100


In [3]:
oxides = pd.DataFrame([[60.0855,101.963,94.1966,56.087,79.867,70.938,71.845,61.9796,40.305,141.9476,151.990,74.693,64.066,153.326,149.881,265.81,81.379,18.998,35.453,80.063],
                       [2,3,1,1,2,1,1,1,1,5,3,1,2,1,3,5,1,1,1,3],
                       [1/2,2/3,2,1,1/2,1,1,2,1,2/5,2/3,1,1/2,1,2/3,2/5,1,1,1,1/3]],
                      columns=['sio2','al2o3','k2o','cao','tio2','mno','feo','na2o','mgo','p2o5','cr2o3','nio','so2','bao','v2o3','nb2o5','zno','f','cl','so3'])
spots = fayl.iloc[:,0]
data = fayl.iloc[:,1:]
data[data < 0] = 0

newframe = pd.concat([data,oxides], join='inner')
newframe = pd.concat([newframe.iloc[:-3,:].div(newframe.iloc[-3,:]),newframe.iloc[-3:]]).reset_index().drop('index',axis=1).add_suffix('_molar proportion')

atomico2 = newframe.iloc[:-3,:].multiply(newframe.iloc[-2,:])
atomico2['number o2-'] = atomico2.sum(axis=1)
atomico2['normalizing factor'] = 4 / atomico2['number o2-']
atomico2 = pd.concat([atomico2,newframe.iloc[-3:]])

anions = atomico2.iloc[:-3,:-2].multiply(atomico2.iloc[:-3,-1],axis=0)
anions['negative charge total'] = anions.sum(axis=1)
anions = pd.concat([anions,newframe.iloc[-3:]])
anions.columns = anions.columns.str.replace('_molar proportion','_charge distribution')

cations = anions.iloc[:-3,:-1].multiply(anions.iloc[-1,:-1])
cations['sum cations'] = cations.sum(axis=1)
cations.columns = cations.columns.str.replace('_charge distribution','_as cation')
cations['M sites Total'] = cations['cao_as cation']+cations['mgo_as cation']+cations['feo_as cation']+cations['mno_as cation']
cations['Fa'] = cations['feo_as cation'] / cations['M sites Total']
cations['Fo'] = cations['mgo_as cation'] / cations['M sites Total']
cations['Tp'] = cations['mno_as cation'] / cations['M sites Total']
cations['Mo'] = cations['cao_as cation'] / cations['M sites Total']

cations['Fo error propagated'] = cations['Fo'].mean() * ((stdev(cations['mgo_as cation'])/(cations['mgo_as cation'].mean()))**2 + (((stdev(cations['cao_as cation']))**2+(stdev(cations['mgo_as cation'])**2)+(stdev(cations['feo_as cation'])**2)+(stdev(cations['mno_as cation'])**2))**(1/2)/(cations['M sites Total'].mean()))**2)**(1/2)
cations['Fa error propagated'] = cations['Fa'].mean() * ((stdev(cations['feo_as cation'])/(cations['feo_as cation'].mean()))**2 + (((stdev(cations['cao_as cation']))**2+(stdev(cations['mgo_as cation'])**2)+(stdev(cations['feo_as cation'])**2)+(stdev(cations['mno_as cation'])**2))**(1/2)/(cations['M sites Total'].mean()))**2)**(1/2)

olivine_recalc = pd.concat([spots,data,atomico2,anions,cations],axis=1).fillna(0)

anions.head(50)


,sio2_charge distribution,al2o3_charge distribution,mno_charge distribution,feo_charge distribution,na2o_charge distribution,cao_charge distribution,tio2_charge distribution,cr2o3_charge distribution,nio_charge distribution,mgo_charge distribution,negative charge total
0,1.961764,0.011473,0.064654,1.080079,0.000899,0.048005,0.004650,0.000733,0.001243,0.826500,4.0
1,1.955931,0.017015,0.064690,1.110777,0.000000,0.044236,0.005606,0.001473,0.000499,0.799774,4.0
2,1.935805,0.013742,0.078480,1.176902,0.000603,0.043635,0.006082,0.000737,0.001000,0.743014,4.0
3,1.932284,0.013249,0.075909,1.179886,0.001211,0.044157,0.004698,0.000741,0.000251,0.747613,4.0
4,1.955093,0.018979,0.060534,1.079711,0.000892,0.044032,0.006461,0.001091,0.000987,0.832219,4.0
5,1.939118,0.020320,0.065519,1.137702,0.000903,0.044263,0.007011,0.001105,0.000000,0.784058,4.0
6,1.957369,0.018354,0.061558,1.086389,0.000592,0.042854,0.011027,0.001086,0.000000,0.820770,4.0
7,1.949810,0.019542,0.061899,1.089337,0.000595,0.043421,0.010164,0.000364,0.000000,0.824867,4.0
8,1.953014,0.027008,0.052277,0.998611,0.001185,0.043861,0.009195,0.000362,0.001229,0.913259,4.0
9,1.960590,0.018448,0.063431,1.064459,0.000298,0.049320,0.008774,0.001456,0.000494,0.832730,4.0


In [4]:
def recalc_olivine(path,sheet_name):
    """
    READ ME
    
    This will recalculate olivine analysis and propagate errors with a standard error analysis for Fo and Fa
    
    Inputs:
        path = some path name to your file. This must be input in parentheses and end in .xlsx
        Your file headers must be in all lower case. For instance, SiO2 must be sio2, etc
        Your first column must be the name that you gave your spot when measuring
        sheet_name = input the sheet name of that excel file where your data lives. This must also be in parentheses. 
        
    """
    #import dependables
    import pandas as pd
    import numpy as np
    import math
    from statistics import stdev
    df = pd.read_excel(path,sheet_name)
    #set up a general dataframe that has a number of oxides, their number of O2-, the cation:anion, and the molecular weights
    oxides = pd.DataFrame([[60.0855,101.963,94.1966,56.087,79.867,70.938,71.845,61.9796,40.305,141.9476,151.990,74.693,64.066,153.326,149.881,265.81,81.379,18.998,35.453,80.063],
                          [2,3,1,1,2,1,1,1,1,5,3,1,2,1,3,5,1,1,1,3],
                          [1/2,2/3,2,1,1/2,1,1,2,1,2/5,2/3,1,1/2,1,2/3,2/5,1,1,1,1/3]],
                      columns=['sio2','al2o3','k2o','cao','tio2','mno','feo','na2o','mgo','p2o5','cr2o3','nio','so2','bao','v2o3','nb2o5','zno','f','cl','so3'])
    #pull out the column with strings
    spots = df.iloc[:,0]
    #pull out all the columns with data in them
    data = df.iloc[:,1:]
    #set any negative numbers = 0
    data[data < 0] = 0
    #put together the measured data and the general oxides dataframe, dropping any columns from 'oxides' that does not exist in the measured dataset 
    newframe = pd.concat([data,oxides], join='inner')
    
    #the line below does a few things:
    #1. gives you the molar proportion of oxides by dividing measurements by molecular weights
    #2. reorganizes the dataframes to retain the oxides dataframe in a neat manner
    #3. adds a suffix for the eventual output dataframe accordingly
    newframe = pd.concat([newframe.iloc[:-3,:].div(newframe.iloc[-3,:]),newframe.iloc[-3:]]).reset_index().drop('index',axis=1).add_suffix('_molar proportion')

    #multiply by the number of oxygens per oxide in order to get the atomic proportion of oxygen
    atomico2 = newframe.iloc[:-3,:].multiply(newframe.iloc[-2,:])
    #add up the atomic proportion of oxygen. This helps you normalize your measurement in a step below
    atomico2['number o2-'] = atomico2.sum(axis=1)
    #make a normalizing factor for each analysis based on an ideal number of oxygen with respect to what you actually measured
    atomico2['normalizing factor'] = 4 / atomico2['number o2-']
    #put the oxides information back in for ease of calculating and cleanliness
    atomico2 = pd.concat([atomico2,newframe.iloc[-3:]])

    #get the amount of negative charge that is required for each of the cations you measured. This is effectively how your ideal number of negative charges (which you've obtained by normalizing) are distributed about your mineral site based on the cations that are actually measureable.
    anions = atomico2.iloc[:-3,:-2].multiply(atomico2.iloc[:-3,-1],axis=0)
    #sum the total number of negative charges. This should sum up to the numerator you have used above. For the case of olivine, this would be 4.
    anions['negative charge total'] = anions.sum(axis=1)
    #put the oxides information back in for ease of calculating and cleanliness
    anions = pd.concat([anions,newframe.iloc[-3:]])
    #change the suffix at the end of this series of columns for outputting later
    anions.columns = anions.columns.str.replace('_molar proportion','_charge distribution')
    
    #multiply the distributed oxygen charge by the ratio of cation to anion. A simple unit analysis should guide your intution
    cations = anions.iloc[:-3,:-1].multiply(anions.iloc[-1,:-1])
    #sum the number of cations. This should be pretty close to your mineral stoichiometry. For olivine, that is 3 (i.e., (Fe,Mg)2SiO4)
    cations['sum cations'] = cations.sum(axis=1)
    #change the suffix for hte purposes of outputting later
    cations.columns = cations.columns.str.replace('_charge distribution','_as cation')
    #calculate the sum of those cations that contribute to the endmember of the mineral
    cations['M sites Total'] = cations['cao_as cation']+cations['mgo_as cation']+cations['feo_as cation']+cations['mno_as cation']
    #Fa content
    cations['Fa'] = cations['feo_as cation'] / cations['M sites Total']
    #Fo content
    cations['Fo'] = cations['mgo_as cation'] / cations['M sites Total']
    #Tp content
    cations['Tp'] = cations['mno_as cation'] / cations['M sites Total']
    #Mo content
    cations['Mo'] = cations['cao_as cation'] / cations['M sites Total']

    #use error propagation to get the error on your Fo. If you are the type to monitor your standards, you could use this to get your analytical error. Note this does NOT include error of individual analyses and that is for you to decide whether to use or not; this is simply an error analysis based on your standard deviation. 
    cations['Fo error propagated'] = cations['Fo'].mean() * ((stdev(cations['mgo_as cation'])/(cations['mgo_as cation'].mean()))**2 + (((stdev(cations['cao_as cation']))**2+(stdev(cations['mgo_as cation'])**2)+(stdev(cations['feo_as cation'])**2)+(stdev(cations['mno_as cation'])**2))**(1/2)/(cations['M sites Total'].mean()))**2)**(1/2)
    #as above, but for Fa
    cations['Fa error propagated'] = cations['Fa'].mean() * ((stdev(cations['feo_as cation'])/(cations['feo_as cation'].mean()))**2 + (((stdev(cations['cao_as cation']))**2+(stdev(cations['mgo_as cation'])**2)+(stdev(cations['feo_as cation'])**2)+(stdev(cations['mno_as cation'])**2))**(1/2)/(cations['M sites Total'].mean()))**2)**(1/2)

    #concatenate all the above information and fill non-existing values with a 0
    olivine_recalc = pd.concat([spots,data,atomico2,anions,cations],axis=1).fillna(0)
    #output the dataframe to an excel file
    output = olivine_recalc.to_excel('Olivine_recalculated.xlsx')
    
    return output

In [5]:
recalc_olivine('/Users/ctlewis/Documents/Projects/Newberry maybe/Data/Phases/Phases_new_useful/Lewis_051321_fayalite.xlsx',
               'filtered')